Table as_16_q: Asylum seekers in receipt of Section 95 support, by local authority, as at end of quarter

In [1]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 4']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='as_16_q', library='pyexcel-ods3')

In [2]:
df = pd.DataFrame(data.get_array())

In [3]:
observations = df.iloc[3:, :7]
observations.rename(columns= observations.iloc[0], inplace=True)
observations.drop(observations.index[0], inplace = True)
observations.drop('Region', axis = 1, inplace = True)
Final_table = pd.melt(observations,
                       ['Quarter','Local Authority'],
                       var_name="Total supported under Section 95",
                       value_name="Value")
Final_table.Value.dropna(inplace =True)
Final_table['Unit'] = 'Asylum Seekers'
Final_table['Measure Type'] = 'Count'

In [4]:
Final_table['Value'] = Final_table['Value'].map(lambda x : ''
                                                   if (x == '.') | (x == 'z') | ( x == ':')
                                                    else x )

In [5]:
import numpy as np
Final_table['Value'].replace('', np.nan, inplace=True)
Final_table.dropna(subset=['Value'], inplace=True)
Final_table['Value'] = Final_table['Value'].apply(lambda x: pd.to_numeric(x, downcast='integer'))
Final_table['Value'] = Final_table['Value'].astype(int)

In [6]:
Final_table.rename(columns={'Quarter': 'Period'}, inplace=True)
Final_table.rename(columns={'Total supported under Section 95': 'Section 95 Support'}, inplace=True)
Final_table['Period'] = Final_table['Period'].astype(str)
Final_table = Final_table[Final_table['Period'] != '']
Final_table['Period'] = 'quarter/' + Final_table['Period'].map(lambda cell: cell.replace(' ','-'))
Final_table['Section 95 Support'] = Final_table['Section 95 Support'].map(lambda cell: cell.replace('In receipt of subsistence \nonly', 'In receipt of subsistence only'))

In [7]:
Final_table['Local Authority'] = Final_table['Local Authority'].str.lstrip('*')

In [8]:
Final_table['Local Authority'] = Final_table['Local Authority'].map(
    lambda x: {
        'Total East Midlands':'East Midlands',
        'Total East of England':'East of England',
        'Total London':'London',
        'Total North East':'North East',
        'Total North West':'North West',
        'Total Northern Ireland':'Northern Ireland',
        'Total Scotland':'Scotland',
        'Total South East':'South East',
        'Total South West':'South West',
        'Total Wales':'Wales',
        'Total West Midlands':'West Midlands',
        'Total Yorkshire and The Humber':'Yorkshire and The Humber',
        'Total England' : 'England',
        'Total Other and Unknown' : 'Other and Unknown'
        }.get(x, x))

In [9]:
import urllib.request as request
import csv
import io
import requests
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/HO-asylum/master/gss-codes.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/HO-asylum/master/gss-codes.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
Final_table = pd.merge(Final_table, c, how = 'left', left_on = 'Local Authority', right_on = 'Label')
Final_table.columns = ['Geography' if x=='Code' else x for x in Final_table.columns]

In [10]:
Final_table = Final_table[['Period','Geography','Section 95 Support','Measure Type','Value','Unit']]

In [11]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.to_csv(destinationFolder / ('as_16_q.csv'), index = False)

In [12]:
Final_table

,Period,Geography,Section 95 Support,Measure Type,Value,Unit
0,quarter/2003-Q4,K02000001,Total supported under Section 95,Count,80123,Asylum Seekers
1,quarter/2003-Q4,E12000004,Total supported under Section 95,Count,4524,Asylum Seekers
2,quarter/2003-Q4,E12000006,Total supported under Section 95,Count,1944,Asylum Seekers
3,quarter/2003-Q4,E12000007,Total supported under Section 95,Count,24916,Asylum Seekers
4,quarter/2003-Q4,E12000001,Total supported under Section 95,Count,5172,Asylum Seekers
5,quarter/2003-Q4,E12000002,Total supported under Section 95,Count,9093,Asylum Seekers
6,quarter/2003-Q4,N92000002,Total supported under Section 95,Count,153,Asylum Seekers
7,quarter/2003-Q4,S92000003,Total supported under Section 95,Count,5818,Asylum Seekers
8,quarter/2003-Q4,E12000008,Total supported under Section 95,Count,3208,Asylum Seekers
9,quarter/2003-Q4,E12000009,Total supported under Section 95,Count,1668,Asylum Seekers
